# Prepare Environment

In [255]:
%pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [256]:
%pip install pandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [257]:
%pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [402]:
import pandas as pd
pd.set_option('display.max_rows', 500)  # Set the maximum number of rows to display
pd.set_option('display.max_columns', 500)  # Set the maximum number of columns to display
pd.set_option('display.width', 1000)  # Set the overall display width




# Load Data
The first step is to load the data from the train.csv file and split it into train and verification sets

In [403]:
# Load the CSV file into a pandas DataFrame
import pandas as pd
from sklearn.model_selection import train_test_split

def load_data(file_name) -> pd.DataFrame:
    # Load the data from the 'train.csv' file
    df = pd.read_csv(file_name)
    return df

def split_data(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    # Split the data into training and verification sets
    train, verification = train_test_split(df, test_size=0.25, random_state=42)
    return train, verification

def print_summary(train: pd.DataFrame, verification: pd.DataFrame) -> None:
    # Print the shapes of the training and verification sets
    print("Training set shape:", train.shape)
    print("Verification set shape:", verification.shape)
    print("First row of training set:")
    print(train)
    print("First row of verification set:")
    print(verification.iloc[0])

# Verify the shapes of the training and verification sets
# if __name__ == "__main__":
#     train, verification = get_train_verification_data()
#     print_summary(train, verification)


# Get Dependent Value



In [404]:
def get_dependent_datasets(train: pd.DataFrame, verification: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    train_dependent = train[['SalePrice']]
    verification_dependent = verification[['SalePrice']]
    return train_dependent, verification_dependent

def remove_dependent_column(train: pd.DataFrame, verification: pd.DataFrame) -> None:
    train.drop('SalePrice', axis=1, inplace=True)  # Remove the dependent column from the training features
    verification.drop('SalePrice', axis=1, inplace=True)  # Remove the dependent column from the verification features


# # Build Features

In [405]:
import pandas as pd

def analyze_features(df):
    """
    Analyzes features of a pandas dataframe.

    Parameters:
    - df: pandas.DataFrame

    Prints:
    - Feature name
    - Percentage of NaN or None values
    - Value distribution summary (for numeric features)
    - Unique values (for categorical features with fewer than 10 unique values)
    """
    for column in df.columns:
        # NaN or None percentages
        nan_percentage = df[column].isnull().mean() * 100
        
        print(f"Feature: {column}")
        print(f"Percentage of NaN/None values: {nan_percentage:.2f}%")
        
        # If numeric, show distribution summary
        if pd.api.types.is_numeric_dtype(df[column]):
            print("Value distribution summary:")
            print(df[column].describe())
        else:
            # For categorical data, if fewer than 10 unique values, show them
            unique_values = df[column].unique()
            if len(unique_values) <= 10:
                print("Unique values:")
                print(unique_values)
        print("-" * 40)


In [538]:
import datetime
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from typing import List

import pandas as pd
from sklearn.preprocessing import OneHotEncoder

import pandas as pd

def one_hot_encode_with_fallback(source: pd.DataFrame, target: pd.DataFrame, feature: str, all_categories: list) -> pd.DataFrame:
    """
    Perform one-hot encoding on a dataframe for a specific feature. This function ensures that
    columns for all possible categories are created, even if some categories are not present in
    the current dataframe. This is useful for preparing both training and test dataframes.

    Parameters:
    - df: The dataframe containing the feature to encode.
    - feature: The name of the categorical feature to encode.
    - all_categories: A list of all possible categories for the feature.

    Returns:
    - A new dataframe with the original feature replaced by one-hot encoded columns.
    """
    
    # Create a temporary DataFrame with the feature to ensure it has a category type with all possible categories
    temp_df = pd.DataFrame({feature: pd.Categorical(source[feature], categories=all_categories)})
    
    # Use pd.get_dummies to perform one-hot encoding
    encoded_df = pd.get_dummies(temp_df, prefix=feature).astype('int')
    
    # Drop the original feature from the input dataframe and concatenate the new encoded_df
    target = pd.concat([target, encoded_df], axis=1)
    
    return target

def add_one_hot_features(source: pd.DataFrame, target: pd.DataFrame, feature: str) -> pd.DataFrame:
    if feature in source.columns:
        encoded_features = OneHotEncoder(handle_unknown='ignore',sparse_output=False)\
            .set_output(transform='pandas')\
            .fit_transform(source[[feature]])
        target = pd.concat([target, encoded_features], axis=1)
    return target

def normalize_feature_MinMax(source: pd.DataFrame, target: pd.DataFrame, feature: str) -> pd.DataFrame:
    if feature in source.columns:
        scaler = MinMaxScaler( ).set_output(transform='pandas')
        encoded_feature = scaler.fit_transform(source[[feature]])
        target = pd.concat([target, encoded_feature], axis=1)
    return target

def normalize_feature_Standard(source: pd.DataFrame, target: pd.DataFrame, feature: str) -> pd.DataFrame:
    if feature in source.columns:
        scaler = StandardScaler( ).set_output(transform='pandas')
        encoded_feature = scaler.fit_transform(source[[feature]])
        target = pd.concat([target, encoded_feature], axis=1)
    return target

def ordinal_encode_normalize(source: pd.DataFrame, target: pd.DataFrame, feature: str, categories: List[str]) -> pd.DataFrame:
    if feature in source.columns:
        encoded_feature = source[feature].apply(lambda x: categories.index(x) if x in categories else x)
        target = pd.concat([target, encoded_feature], axis=1)
        scaler = MinMaxScaler( ).set_output(transform='pandas')
        encoded_feature = scaler.fit_transform(target[[feature]] )
        target.drop(feature, axis=1, inplace=True)
        target = pd.concat([target, encoded_feature], axis=1)

    return target

def add_age_features(source: pd.DataFrame, target: pd.DataFrame, feature: str, range: int) -> pd.DataFrame:
    
    if feature not in source.columns:
        return source
    
    current_year = datetime.datetime.now().year
    target[feature + '_age'] = 100 - (current_year - source[feature]) / 100
    
    # new_feature_name = f'{feature}_{range}_category'

    # source[new_feature_name ] = source[feature + '_age'] // range
    # source = one_hot_encode_with_fallback(source, new_feature_name, ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'])
    # source.drop(new_feature_name , axis=1, inplace=True)
    # source.drop(feature + '_age', axis=1, inplace=True)
    
    return target

def add_feature(source: pd.DataFrame, target: pd.DataFrame, feature: str) -> pd.DataFrame:
    target[str] = source[feature]
    return target

def build_features(source: pd.DataFrame) -> pd.DataFrame:
    
    target = pd.DataFrame()        
    #features = add_one_hot_features(source, features, 'MSSubClass')
    target = one_hot_encode_with_fallback(source, target, 'MSZoning', ['A', 'C', 'FV', 'I', 'RH', 'RL', 'RP', 'RM'])
    #features = normalize_feature_Standard(source, features, 'LotFrontage')
    target = normalize_feature_Standard(source, target, 'LotArea')
    #features = add_one_hot_features(source, features, 'Street')
    #features = add_one_hot_features(source, features, 'Alley')
    #features = add_one_hot_features(source, features, 'LotShape')
    #features = add_one_hot_features(source, features, 'LandContour')
    #features = add_one_hot_features(source, features, 'Utilities')
    #features = add_one_hot_features(source, features, 'LotConfig')
    #features = add_one_hot_features(source, features, 'LandSlope')
    target = one_hot_encode_with_fallback(source, target, 'Neighborhood', ['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr', 'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel', 'NAmes', 'NoRidge', 'NPkVill', 'NridgHt', 'NWAmes', 'OldTown', 'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber', 'Veenker'])
    #features = add_one_hot_features(source, features, 'Condition1')
    #features = add_one_hot_features(source, features, 'Condition2')
    target = one_hot_encode_with_fallback(source, target, 'BldgType', ['1Fam', '2fmCon', 'Duplex', 'Twnhs', 'TwnhsE'])
    #features = add_one_hot_features(source, features, 'HouseStyle')
    #features = normalize_feature(source, features, 'OverallQual')
    
    target = one_hot_encode_with_fallback(source, target, 'OverallCond', ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
    #features = add_one_hot_features(source, features, 'OverallCond')
    
    #features = add_age_features(source, features, 'YearBuilt')
    target = add_age_features(source, target, 'YearRemodAdd', 10)
    #features = add_one_hot_features(source, features, 'RoofStyle')
    #features = add_one_hot_features(source, features, 'RoofMatl')
    #features = add_one_hot_features(source, features, 'Exterior1st')
    #features = add_one_hot_features(source, features, 'Exterior2nd')
    # features = add_one_hot_features(source, features, 'MasVnrType')
    # features = normalize_feature(source, features, 'MasVnrArea')
    #features = ordinal_encode_normalize(source, features, 'ExterQual', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])

    target = one_hot_encode_with_fallback(source, target, 'ExterCond', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
    #features = ordinal_encode_normalize(source, features, 'ExterCond', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])

    target = one_hot_encode_with_fallback(source, target, 'Foundation', ['BrkTil', 'CBlock', 'PConc', 'Slab', 'Stone', 'Wood'])
    #features = ordinal_encode_normalize(source, features, 'BsmtQual', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
    #features = ordinal_encode_normalize(source, features, 'BsmtCond', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
    #features = ordinal_encode_normalize(source, features, 'BsmtExposure', ['No', 'Mn', 'Av', 'Gd'])
    #features = ordinal_encode_normalize(source, features, 'BsmtFinType1', ['No', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'])
    #features = normalize_feature(source, features, 'BsmtFinSF1')
    target = one_hot_encode_with_fallback(source, target, 'BsmtFinType2', ['No', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'])
    #features = normalize_feature(source, features, 'BsmtFinSF2')
    #features = normalize_feature(source, features, 'BsmtUnfSF')
    target = normalize_feature_Standard(source, target, 'TotalBsmtSF')
    #features = add_one_hot_features(source, features, 'Heating')
    #features = ordinal_encode_normalize(source, features, 'HeatingQC', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
    
    target = one_hot_encode_with_fallback(source, target, 'CentralAir', ['N', 'Y'])
    #features = ordinal_encode_normalize(source, features, 'CentralAir', ['N', 'Y'])
    
    #features = add_one_hot_features(source, features, 'Electrical')
    #features = normalize_feature(source, features, '1stFlrSF')
    #features = normalize_feature(source, features, '2ndFlrSF')
    #features = normalize_feature(source, features, 'LowQualFinSF')
    target = normalize_feature_Standard(source, target, 'GrLivArea')
    #features = normalize_feature(source, features, 'BsmtFullBath')
    #features = normalize_feature(source, features, 'BsmtHalfBath')
    target = one_hot_encode_with_fallback(source, target, 'FullBath', ['0', '1', '2', '3'])
    #features = add_one_hot_features(source, features, 'HalfBath')

    target = normalize_feature_Standard(source, target, 'BedroomsAbvGr')
    #source = one_hot_encode_with_fallback(source, 'BedroomsAbvGr', ['0', '1', '2', '3', '4', '5', '6', '8'])

    #features = add_one_hot_features(source, features, 'KitchenAbvGr')
    
    target = one_hot_encode_with_fallback(source, target, 'KitchenQual', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
    #features = ordinal_encode_normalize(source, features, 'KitchenQual', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
    
    target = one_hot_encode_with_fallback(source, target, 'TotRmsAbvGrd', ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '14'])
    #features = ordinal_encode_normalize(source, features, 'Functional', ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'])
    #features = add_one_hot_features(source, features, 'Fireplaces')
    #features = ordinal_encode_normalize(source, features, 'FireplaceQu', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])    
    #features = add_one_hot_features(source, features, 'GarageType')
    #features = add_age_features(source, features, 'GarageYrBlt')
    #features = ordinal_encode_normalize(source, features, 'GarageFinish', ['Unf', 'RFn', 'Fin'])   
    
    target = one_hot_encode_with_fallback(source, target, 'GarageCars', [0,1,2,3,4])
    
    #features = normalize_feature(source, features, 'GarageArea')
    #features = ordinal_encode_normalize(source, features, 'GarageQual', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])

    #features = add_one_hot_features(source, features, 'GarageCond')
    
    #features = add_one_hot_features(source, features, 'PavedDrive')
    #features = ordinal_encode_normalize(source, features, 'PavedDrive', ['N', 'P', 'Y'])
    
    #features = normalize_feature(source, features, 'WoodDeckSF')
    #features = normalize_feature(source, features, 'OpenPorchSF')
    #features = normalize_feature(source, features, 'EnclosedPorch')
    #features = normalize_feature(source, features, '3SsnPorch')
    #features = normalize_feature(source, features, 'ScreenPorch')
    #features = normalize_feature(source, features, 'PoolArea')
    
    #features = add_one_hot_features(source, features, 'PoolQC')
    #features = ordinal_encode_normalize(source, features, 'PoolQC', ['Po', 'Fa', 'TA', 'Gd', 'Ex'])
    
    #features = ordinal_encode_normalize(source, features, 'Fence', ['MnWw', 'GdWo', 'MnPrv', 'GdPrv'])
    #features = add_one_hot_features(source, features, 'MiscFeature')
    #features = normalize_feature(source, features, 'MiscVal')
    #features = add_one_hot_features(source, features, 'MoSold')
    
    #features = add_age_features(source, features, 'YrSold', 5)
    
    #features = add_one_hot_features(source, features, 'SaleType')
    #features = add_one_hot_features(source, features, 'SaleCondition')

    return target

def compare_sets(train_features: pd.DataFrame, verification_features: pd.DataFrame) -> None:
    # Compare the features in the training and verification sets
    train_features_set = set(train_features.columns)
    verification_features_set = set(verification_features.columns)
    features_only_in_train = train_features_set - verification_features_set
    features_only_in_verification = verification_features_set - train_features_set
    print("Features only in train_features:")
    print(features_only_in_train)
    print("Features only in verification_features:")
    print(features_only_in_verification)
    
def make_training_data() -> pd.DataFrame:
    # Load the data from the 'train.csv' file
    training_data = load_data('train.csv')
    features = build_features(training_data)
    features['SalePrice'] = training_data['SalePrice']
    
    train_features, verification_features = split_data(features)
    train_dependent, verification_dependent = get_dependent_datasets(train_features, verification_features)
    #train_features, verification_features = remove_dependent_column(train_features, verification_features)

    train_features.drop('SalePrice', axis=1, inplace=True)  # Remove the dependent column from the training features
    verification_features.drop('SalePrice', axis=1, inplace=True)  # Remove the dependent column from the verification features
    return train_features, verification_features, train_dependent, verification_dependent

if __name__ == "__main__":

    train_features, verification_features, train_dependent, verification_dependent = make_training_data()
    analyze_features(train_features)
    #print(train_features.describe(include='all').transpose()) 
                        #print("train_features:", train_features.iloc[0].to_frame())
    #print("verification_features: ", verification_features.head(1))
    #compare_sets(train_features, verification_features)
    #print_summary(train_features, verification_features)
    


    

Feature: MSZoning_A
Percentage of NaN/None values: 0.00%
Value distribution summary:
count    1095.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: MSZoning_A, dtype: float64
----------------------------------------
Feature: MSZoning_C
Percentage of NaN/None values: 0.00%
Value distribution summary:
count    1095.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: MSZoning_C, dtype: float64
----------------------------------------
Feature: MSZoning_FV
Percentage of NaN/None values: 0.00%
Value distribution summary:
count    1095.000000
mean        0.042922
std         0.202775
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: MSZoning_FV, dtype: float64
----------------------------------------
Feature: MSZoning_I
Percentage of NaN/None values: 0.00%
Value distribution summary:
count    109

# Build model

In [510]:
model.summary()

if __name__ == "__main__":
    
    test_data = load_data('test.csv')
    features = build_features(test_data)
    print('Shape: ', features.shape)
    print(features.head().transpose())
    #test_predictions = model.predict(features)
    #print(test_predictions)
    compare_sets(train_features, features)



Model: "sequential_156"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_589 (Dense)               │ (None, 128)            │        20,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_590 (Dense)               │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_591 (Dense)               │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_592 (Dense)               │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,305 (118.38 KB)

 Trainable params: 30,305 (118.38 KB)

 Non-trainable params: 0 (0.00 B)

Shape:  (1459, 81)
                        0        1        2        3        4
Id                   1461     1462     1463     1464     1465
MSSubClass             20       20       60       60      120
MSZoning               RH       RL       RL       RL       RL
LotFrontage          80.0     81.0     74.0     78.0     43.0
LotArea             11622    14267    13830     9978     5005
Street               Pave     Pave     Pave     Pave     Pave
Alley                 NaN      NaN      NaN      NaN      NaN
LotShape              Reg      IR1      IR1      IR1      IR1
LandContour           Lvl      Lvl      Lvl      Lvl      HLS
Utilities          AllPub   AllPub   AllPub   AllPub   AllPub
LotConfig          Inside   Corner   Inside   Inside   Inside
LandSlope             Gtl      Gtl      Gtl      Gtl      Gtl
Neighborhood        NAmes    NAmes  Gilbert  Gilbert  StoneBr
Condition1          Feedr     Norm     Norm     Norm     Norm
Condition2           Norm     Norm     Norm     Nor

In [534]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam, Ftrl


def build_regression_model(input_shape):
    model = keras.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Dense(128, activation='relu', input_shape=input_shape),
        layers.Dense(64, activation='relu'),
        layers.Dense(16, activation='relu'),
        layers.Dense(1)
    ])

    model.compile(optimizer=Adam(learning_rate=.025), loss='mean_squared_error',metrics=['mae'])

    return model


In [541]:
train_features, verification_features, train_dependent, verification_dependent = make_training_data()
#analyze_features(train_features)
print(train_features.head(5).transpose())

    
print("Train features shape:", train_features.shape)
print("Verification features shape:", verification_features.shape)
print("Train dependent shape:", train_dependent.shape)
print("Verification dependent shape:", verification_dependent.shape)

model = build_regression_model(train_features.columns.shape)
#print(model.summary())
model.fit(train_features, train_dependent, epochs=60, batch_size=100, verbose=2, validation_data=(verification_features, verification_dependent))


                           1023       810        1384       626        813 
MSZoning_A             0.000000   0.000000   0.000000   0.000000   0.000000
MSZoning_C             0.000000   0.000000   0.000000   0.000000   0.000000
MSZoning_FV            0.000000   0.000000   0.000000   0.000000   0.000000
MSZoning_I             0.000000   0.000000   0.000000   0.000000   0.000000
MSZoning_RH            0.000000   0.000000   0.000000   0.000000   0.000000
MSZoning_RL            1.000000   1.000000   1.000000   1.000000   1.000000
MSZoning_RP            0.000000   0.000000   0.000000   0.000000   0.000000
MSZoning_RM            0.000000   0.000000   0.000000   0.000000   0.000000
LotArea               -0.735111  -0.037766  -0.146006   0.182922  -0.076853
Neighborhood_Blmngtn   1.000000   0.000000   0.000000   0.000000   0.000000
Neighborhood_Blueste   0.000000   0.000000   0.000000   0.000000   0.000000
Neighborhood_BrDale    0.000000   0.000000   0.000000   0.000000   0.000000
Neighborhood

C:\Users\benja\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
C:\Users\benja\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 - 1s - 116ms/step - loss: 38358663168.0000 - mae: 179777.2812 - val_loss: 35382591488.0000 - val_mae: 168873.2188
Epoch 2/60
11/11 - 0s - 7ms/step - loss: 25389000704.0000 - mae: 136732.8750 - val_loss: 8745679872.0000 - val_mae: 60264.1953
Epoch 3/60
11/11 - 0s - 6ms/step - loss: 7812467712.0000 - mae: 71079.8516 - val_loss: 6160331264.0000 - val_mae: 58463.7266
Epoch 4/60
11/11 - 0s - 8ms/step - loss: 5781499904.0000 - mae: 51353.4336 - val_loss: 5948686336.0000 - val_mae: 51383.3750
Epoch 5/60
11/11 - 0s - 7ms/step - loss: 5022448128.0000 - mae: 53034.3516 - val_loss: 5411750912.0000 - val_mae: 55826.3672
Epoch 6/60
11/11 - 0s - 7ms/step - loss: 4586080256.0000 - mae: 47709.2148 - val_loss: 4874140160.0000 - val_mae: 48608.4336
Epoch 7/60
11/11 - 0s - 6ms/step - loss: 4121350400.0000 - mae: 47173.3984 - val_loss: 4427247616.0000 - val_mae: 47469.7422
Epoch 8/60
11/11 - 0s - 6ms/step - loss: 3714463488.0000 - mae: 42002.9570 - val_loss: 3974086912.0000 - val_mae: 44912.4805
Epo

# Predict Prices

In [572]:
import pandas as pd
#model.summary()

if __name__ == "__main__":
    
    test_data = load_data('test.csv')
    features = build_features(test_data)
    #print('Shape: ', features.shape)
    #print(features.head().transpose())
    test_predictions = model.predict(features)
    print(test_predictions.shape)
    print(test_data['Id'].shape)
    # Assuming test_data['id'] is your Pandas Series and test_predictions is your NumPy array
    result_df = pd.DataFrame({
        "ID": test_data['Id'],
        "SalePrice": test_predictions.flatten()
    })

    # Display the first few rows to verify
    print(result_df.head())
    result_df.to_csv('submission.csv', index=False)












46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
(1459, 1)
(1459,)
     ID     SalePrice
0  1461  109245.50000
1  1462  160852.06250
2  1463  190412.78125
3  1464  202717.53125
4  1465  223857.00000
